In this tutorial notebook, we'll go through the basic setup for using prospector to fit some UV spectra. Note that prospector is one of many tools in this space that can work directly with spectra (see also BEAGLE, Bagpipes, [...](http://www.sedfitting.org/Fitting.html)) which all have different strengths and weaknesses. With prospector, we have the advantage of being able to work with both FSPS and BPASS readily (and in principle, other models), which feature quite different treatments of massive star evolution and atmospheres.

# Setting up prospector:
First, make sure you're running in a python environment with 'the basics' (likely you already have these if you're running with conda):
- numpy
- scipy
- astropy
- matplotlib
- h5py

Next you'll need to set up some additional packages:
- emcee or dynesty [pick your poison for domain exploration: MCMC, or nested sampling]
- sedpy (for filters, likely optional here): `pip install astro-sedpy`
- corner for pretty corner plots: `pip install corner`
- *(definitely optional but nice:)* mpi4py for multiprocessing in the sampling step

Next, we need to set up our actual galaxy modeling framework. In principle, this can be even more generally defined to allow you to plug-in your own bespoke complicated model. But for this tutorial and many simple fitting activites, FSPS is great.

Note that we don't actually have to compile FSPS itself (make sure to note the citations in the readme though!); simply clone the repository somewhere convenient and make sure that you point the `$SPS_HOME` environment variable at it in the shell you are running python in (per [the python-fsps documentation](https://dfm.io/python-fsps/current/installation/)):
```
export SPS_HOME="/path/where/you/want/to/download/fsps"
git clone https://github.com/cconroy20/fsps.git $SPS_HOME
```
Then, you just need to install the python-fsps bindings themselves. One great thing about FSPS is that you can change the stellar libraries used; however, this can only be done at compile time. Conveniently, this flexibility is also built into the python-fsps bindings installation procedure: here, let's start with:
```
pip uninstall fsps  # if you already have python-fsps installed
FFLAGS="-DMIST=0 -DPADOVA=0 -DMILES=0 -DBASEL=0 -DBPASS=1" python -m pip install fsps --no-binary fsps
```

Finally, we're ready to install prospector itself: 

`python -m pip install astro-prospector`

# Fetch some data to fit:
Here I've included an example galaxy spectrum from the CLASSY survey, along with some helpful material discussed below. More on the provenance and format of these files can be found [here](https://archive.stsci.edu/hlsp/classy).

In [ ]:
# basic imports and notebook setup
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('notebook')

First, let's open up and take a look at the example CLASSY spectrum we'll be analyzing:

In [ ]:
targnm = 'J0021+0052'

specf = fits.open(f'data/hlsp_classy_hst_cos_{targnm.lower()}_multi_v1_coadded.fits')

# the HR G130M+G160M data, in the rest frame and corrected for Galactic dust extinction:
data_hr = specf['REST HR COADDED + GAL CORR'].data
wlrest_raw = data_hr['wave']
flux_raw = data_hr['flux density']
err_raw = data_hr['error']

# bin by 15 pixels (~0.17 A) to reduce noise
smooth = 15
wlrest = (np.convolve(wlrest_raw, np.ones(smooth), mode='same') / smooth)[smooth:-smooth:smooth]
flux = (np.convolve(flux_raw, np.ones(smooth), mode='same') / smooth)[smooth:-smooth:smooth]
err = (np.sqrt(np.convolve(err_raw**2., np.ones(smooth), mode='same')) / smooth)[smooth:-smooth:smooth]

plt.plot(wlrest,flux)
plt.ylim(0,10) # reduce viewed range; Lya swamps everything otherwise
plt.xlabel('rest wavelength (Å)')
plt.ylabel('Flambda (x10^15 erg/s/cm2/Å)')

Looks like a blue UV continuum! There are a lot of stellar feature here, but they're hard to pick out against the transitions dominated by intervening ISM/outflow/MW absorption and Lya. Generally the first step in fitting UV spectra like this is to create a mask to block-out these and any other features we don't want to include in the fit; results can be very biased otherwise. This can be semi-automated with a linelist, but precise work will still generally require at least checking and refining this mask on an object-by-object or at least spectrograph-by-spectrograph basis. This is a little tedious, so for this example we'll load-up a mask we've already generated for this object:

In [ ]:
maskedges_tab = np.genfromtxt(
            f'data/{targnm}_maskedges.csv', delimiter=',')

mask = np.zeros(wlrest.shape, dtype=bool)

for edges in maskedges_tab:
    mask = mask | ( (wlrest>edges[0]) & (wlrest<edges[1]) )

mask = ~mask    # mask where 1 = use, 0 = masked
mask = mask & np.isfinite(flux+err)

plt.plot(wlrest[mask],flux[mask])
plt.plot(wlrest,flux,zorder=-1,alpha=0.2,color='k')

plt.ylim(np.min(flux[mask]), np.max(flux[mask]))
plt.xlabel('rest wavelength (Å)')
plt.ylabel('Flambda (x10^15 erg/s/cm2/Å)')

Much better! Now we can proceed to a trial fit.

Prospector is controlled fully in python, which is great (if you're into python) as it means you have quite a bit of flexibility in how you load your data in and fit it. This directory includes a minimal working example of the default parameter file adjusted to run a basic fit for this spectrum we just examined. Take a look at this file (`prospect_params.py`) in your favorite text editor to see the basic outline of how it works.

To actually run it, we can either use the command line as-usual for a python script, or (convenient here) run it in debug mode with `--debug` inside a python interpreter or notebook.

You should see this next cell spit-out some information about the model you've built, then updates about the emcee chain progress; and finally, a timestamped output file name which contains the results of the run:

In [ ]:
%run prospect_params.py --emcee --optimize --luminosity_distance 452 --object_redshift 0.0

Hopefully that worked! Now we can load-in the results (stored in an HDF5 file) and make some basic diagnostic plots using handy utilities provided by prospector: Note, for this code to work properly, you need to make sure to reload the correct .h5 file - if you just reran it, unless you changed the defaults to overwrite, you'll have to update this to point to the new file each time!

In [ ]:
import prospect.io.read_results as reader

res, obs, model = reader.results_from("prospector_test_run_~~TIMESTAMP~~_result.h5")
# Trace plots
tfig = reader.traceplot(res)
# Corner figure of posterior PDFs
cfig = reader.subcorner(res)

Hopefully you'll see some clustering of points in these diagrams! We're not going to worry about convergence of the MCMC chain in this quick tutorial, but note that this is an important extra step to assess for science-grade results.

Now, let's take a look at the 'bestfit' and see if it looks any good:

In [ ]:
from prospect.plotting.corner import quantile
best = res["bestfit"]

plt.step(wlrest[mask],flux[mask])
plt.step(wlrest,flux,zorder=-1,alpha=0.2,color='k')

fitwl = res['obs']['wavelength']
bestfl = best['spectrum']#/1.e-15
import astropy.units as u
def convert_to_maggies(flam,wls):
    fl =  flam * u.erg/u.s/u.cm**2/u.angstrom
    fl = fl.to(u.Jy, 
            equivalencies=u.spectral_density(wls*u.angstrom)
            ).value / 3631.
    return fl

def convert_to_flam(maggies,wls):
    fl = maggies * 3631. * u.Jy
    fl = fl.to(u.erg/u.s/u.cm**2/u.angstrom,
               equivalencies=u.spectral_density(wls*u.angstrom)
              ).value
    return fl

print(bestfl)
plt.plot(fitwl, convert_to_flam(bestfl,fitwl)/1.e-15)

plt.ylim(np.min(flux[mask]), np.max(flux[mask]))
plt.xlabel('rest wavelength (Å)')
plt.ylabel('Flambda (x10^15 erg/s/cm2/Å)')

You've just fit a UV spectrum using prospector! Hopefully the result looks vaguely reasonable!

This basic tutorial was designed to get us up and running with a simple example, but we ignored plenty of the uncertainties inherent to this sort of fitting. As one option for the rest of the session, take a look at these ideas for things to improve/experiment with and try your hand at one (or a few!); you will likely find the prospector [documentation](https://prospect.readthedocs.io/en/latest/index.html) quite handy.
- Zoom-in and take a closer look at this fit. What features are well fit? Which aren't? Should we believe the metallicity, etc we derive from this fit? Try restricting the wavelength range fit to exclude or focus-in on just the strong wind lines (NV, CIV): how does this change things?
- In the example, we set up a simple stellar population (single-age) fit; but prospector has a huge amount of flexibility in the SFH approach. Experiment with e.g. a non-parametric star formation history; how different do the fits look? Is the history well-constrained by the spectrum?
- Dust is a key ingredient here. Try out some different parameterizations / priors and see how it affects things.
- We ignored nebular emission in this fit (and indeed, hopefully masked most of it intentionally). What happens when you include it?
- We also neglected to account for the line spread function / resolution matching in this experiment. Try accounting for this with a simple fixed-velocity smoothing model, or a wavelenth-dependent approach (hint: the latter will end up being necessary here in this comparison with BPASS).
- Often data reduction leaves the flux calibration for a spectrum far more uncertain than in this example with HST/COS. Explore how to account for this with prospector, and test out on a readjusted version of this spectrum.
- We set FSPS/prospector up to use the BPASS library; as a comparison exercise, try switching to the FSPS-default MIST models. Can you achieve a similar fit using these libraries?
- (Advanced group project I've been meaning to try:) In principle one can add any SPS models to the FSPS framework and thereby to prospector. Can we get e.g. a Starburst99 grid running in prospector (via FSPS, or directly in prospector)?